Setting the Enviroment

In [5]:
#Loading Dependencies

import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs

In [6]:
#Creating the Soup

base_url = 'https://store.steampowered.com/specials'
html_content = r.get(base_url).content
soup = bs(html_content, 'lxml')

#Exporting the Soup for Readability
with open('steam_store_specials', 'wb') as file:
    file.write(soup.prettify('utf-8'))

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

Nested Tree

In [ ]:
# div tab_content_ctn sub
#     h2 tab_content_title
#     div tab_content_section_ctn
#         a class="tab_item"
#             div class="tab_item_name"
#             div class="discount_pct"
#             div class="discount_original_price"
#             div class="discount_final_price"

Creating the Container

In [ ]:
container = soup.find('div', class_ = 'tab_content_ctn sub')
type(container)

Extracting the Tabs Names

In [ ]:
tabs_names = [c.text for c in container.find_all('h2')]
tabs_names

df = pd.DataFrame(columns=tabs_names)
df

Extracting the Tabs Content

In [ ]:
tabs_content = container.find_all('div', class_ = 'tab_content_section_ctn')
type(tabs_content)

Extracting the Games Details from Each Tab

In [ ]:
games_details = [c.find_all('a', class_="tab_item") for c in tabs_content]
type(games_details)

In [ ]:
#Extracting the Name of Each Game from Each tab

game_name = [[l.find('div', class_="tab_item_name").text for l in g] for g in games_details]
game_name

In [ ]:
#Extracting the Original Price of Each Game from Each tab
game_original_price = [[float((l.find('div', class_="discount_original_price").text).strip('$')) for l in g] for g in games_details]
game_original_price

In [ ]:
#Extracting the Discount % of Each Game from Each tab
game_discount = [[int((l.find('div', class_="discount_pct").text).strip('-%')) for l in g] for g in games_details]
game_discount

In [ ]:
#Extracting the Final Price of Each Game from Each tab
game_final_price = [[float((l.find('div', class_="discount_final_price").text).strip('$')) for l in g] for g in games_details]
game_final_price

In [ ]:
idx = 0
pd_data = [[],[],[],[]]
columns_list = ['Name', 'Original Price', 'Discount', "Final Price"]

for i in range(4):
    pd_data[idx] = pd.DataFrame([game_name[idx], game_original_price[idx], game_discount[idx], game_final_price[idx]], index=columns_list).T
    idx += 1

In [ ]:
pd_data[0]

In [ ]:
pd_data[1]

In [ ]:
pd_data[2]

In [ ]:
pd_data[3]

Exporting the Data

In [ ]:
new_and_trending = pd.DataFrame()
top_sellers = pd.DataFrame()
being_played = pd.DataFrame()
pre_purchase = pd.DataFrame()

new_and_trending['Name'] = game_name[0]
new_and_trending['Original Price'] = game_original_price[0]
new_and_trending['Discount %'] = game_discount[0]
new_and_trending['Final Price'] = game_final_price[0]

top_sellers['Name'] = game_name[1]
top_sellers['Original Price'] = game_original_price[1]
top_sellers['Discount %'] = game_discount[1]
top_sellers['Final Price'] = game_final_price[1]

being_played['Name'] = game_name[2]
being_played['Original Price'] = game_original_price[2]
being_played['Discount %'] = game_discount[2]
being_played['Final Price'] = game_final_price[2]

pre_purchase['Name'] = game_name[3]
pre_purchase['Original Price'] = game_original_price[3]
pre_purchase['Discount %'] = game_discount[3]
pre_purchase['Final Price'] = game_final_price[3]

In [ ]:
new_and_trending

In [ ]:
top_sellers

In [ ]:
being_played

In [ ]:
pre_purchase